In [5]:
import xarray as xr
from os.path import isfile, join
from os import listdir
from google.colab import drive
import torch
import pandas as pd
import numpy as np
import netCDF4 as nc
import datetime
!pip install geopandas
import geopandas as gpd
drive.mount('/content/drive', force_remount=True)

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 16.7 MB 43.7 MB/s 
     |████████████████████████████████| 6.3 MB 40.8 MB/s 
Mounted at /content/drive


In [ ]:
TEST_PERIOD = ("1989-10-01", "1999-09-30")
TRAINING_PERIOD = ("1999-10-01", "2008-09-30")

In [ ]:
def read_single_basin(station_id, basin_boundaries, discharge_file_name, ERA5_data_folder_name, output_folder):

    # get the minimum and maximum longitude and latitude (square boundaries)
    min_lon = np.squeeze(np.floor((basin_boundaries['minx'].values * 10) / 10))
    min_lat = np.squeeze(np.floor((basin_boundaries['miny'].values * 10) / 10))
    max_lon = np.squeeze(np.ceil((basin_boundaries['maxx'].values * 10) / 10))
    max_lat = np.squeeze(np.ceil((basin_boundaries['maxy'].values * 10) / 10))

    # read the discharge of the required station
    df_dis = pd.read_csv(discharge_file_name)
    # convert the columns of year, month, day, hour, minute to datetime and put it as the dataframe index
    df_dis.index = [datetime.datetime(df_dis['year'][i], df_dis['month'][i],
                                      df_dis['day'][i], df_dis['hour'][i],
                                      df_dis['minute'][i]) for i in range(0, len(df_dis))]
    print(df_dis)

    # read the precipitation of the required station
    # ERA5
    year_start = df_dis['year'].min() - 1
    year_end = df_dis['year'].max()
    print(year_start, year_end)
    list_of_dates = []
    list_of_total_precipitations = []
    for year in range(year_start, year_end + 1):
        print(year)
        fn = f"{ERA5_data_folder_name}/tp_US_{year}.nc"
        dataset = nc.Dataset(fn)
        # ti is an array containing the dates as the number of hours since 1900-01-01 00:00
        # e.g. - [780168, 780169, 780170, ...]
        ti = dataset['time'][:]
        if year == year_start:
            lon = dataset['longitude'][:]
            lat = dataset['latitude'][:]
            max_lon_array = lon.max()
            min_lon_array = lon.min()
            max_lat_array = lat.max()
            min_lat_array = lat.min()
            ind_lon_min = np.squeeze(np.argwhere(lon == max(min_lon, min_lon_array)))
            ind_lon_max = np.squeeze(np.argwhere(lon == min(max_lon, max_lon_array)))
            ind_lat_min = np.squeeze(np.argwhere(lat == max(min_lat, min_lat_array)))
            ind_lat_max = np.squeeze(np.argwhere(lat == min(max_lat, max_lat_array)))
        # multiply by 1000 to convert from meter to mm
        tp = np.asarray(dataset['tp'][:, ind_lat_max:ind_lat_min + 1, ind_lon_min:ind_lon_max + 1]) * 1000
        # convert the time to datetime format and append it to the times array
        times = [datetime.datetime.strptime("1900-01-01 00:00", "%Y-%m-%d %H:%M") + datetime.timedelta(hours=int(ti[i]))
                 for i in range(0, len(ti))]
        times = np.asarray(times)
        list_of_dates.append(times)
        list_of_total_precipitations.append(tp)

    # concatenate the datetimes from all the years
    datetimes = np.concatenate(list_of_dates, axis=0)

    # concatenate the percipitation data from all the years
    precip = np.concatenate(list_of_total_precipitations, axis=0)

    print([station_id, lat[ind_lat_min], lat[ind_lat_max], lon[ind_lon_min],
           lon[ind_lon_max], precip.shape])

    df = pd.DataFrame(data=datetimes, index=datetimes)
    datetimes = df.index.to_pydatetime()

    ls = [[precip[i, :, :]] for i in range(0, len(datetimes))]
    df = pd.DataFrame(data=ls, index=datetimes, columns=['precip'])

    # down sample the datetime data into 1D (1 day) bins and sum the values falling into the same bin
    df1 = df.resample('1D').sum()
    datetimes24 = df1.index.to_pydatetime()
    precip24 = np.stack(df1['precip'].values)

    dis = df_dis['flow'].values
    datetimesdis = df_dis.index.to_pydatetime()

    # downsample the datetime data into 1D (1 day) bins and take the mean
    # of the values falling into the same bin
    df1 = df_dis.resample('1D').mean()
    datetimesdis24 = df1.index.to_pydatetime()
    dis24 = np.stack(df1['flow'].values)

    fn = output_folder + '/ERA_5_all_data/shape_' + station_id + '.csv'
    pd.DataFrame(data=[precip24.shape[0], precip24.shape[1], precip24.shape[2]],
                 columns=['time', 'lat', 'lon']).to_csv(fn, index=False)
    fn = output_folder + '/ERA_5_all_data/time24_' + station_id + '.npy'
    np.save(fn, datetimes24)
    fn = output_folder + '/ERA_5_all_data/precip24_' + station_id
    precip24.tofile(fn)
    fn = output_folder + '/ERA_5_all_data/timedis24_' + station_id + '.npy'
    np.save(fn, datetimesdis24)
    fn = output_folder + '/ERA_5_all_data/dis24_' + station_id
    dis24.tofile(fn)
    fn = output_folder + '/ERA_5_all_data/info_' + station_id + '.txt'
    with open(fn, 'w') as f:
        print(precip.shape[0], precip.shape[1], precip.shape[2], lat[ind_lat_min], lat[ind_lat_max], lon[ind_lon_min],
              lon[ind_lon_max], file=f)

    from shapely.geometry import Point

    lonb = lon[ind_lon_min:ind_lon_max]
    latb = lat[ind_lat_max:ind_lat_min]
    lslon = [lonb[i] for i in range(0, len(lonb)) for j in range(0, len(latb))]
    lslat = [latb[j] for i in range(0, len(lonb)) for j in range(0, len(latb))]
    lat_lon_lst = []
    for i in range(0, len(lslon)):
        if np.squeeze(basin['geometry'].contains(Point(lslon[i], lslat[i]))):
            lat_lon_lst.append([lslat[i], lslon[i]])

    fn = output_folder + '/ERA_5_all_data/latlon_' + station_id + '.csv'
    pd.DataFrame(data=lat_lon_lst, columns=['lat', 'lon']).to_csv(fn, index=False, float_format='%6.1f')

    fn = output_folder + '/ERA_5_all_data/info_' + station_id + '.txt'
    with open(fn, 'w') as f:
        print(precip.shape[0], precip.shape[1], precip.shape[2], lat[ind_lat_min], lat[ind_lat_max], lon[ind_lon_min],
              lon[ind_lon_max], file=f)


def preprocess_train_test_data(data):
    data_df = data.to_pandas()
    if len(data["time"].data) > 0:
        data_df["tp"] = data_df["tp"].apply(lambda x: x * 1000)
        data_df = data_df.reset_index()
        data_df["time"] = pd.to_datetime(data_df["time"], infer_datetime_format=True)
        data_df = data_df.resample('1D', on='time').sum()
        data_df = data_df.reset_index()
    return data_df


def preprocess_single_year_file(basin_filename):
    year_dataset = xr.load_dataset(basin_filename)
    print(year_dataset)
    year_dataset = year_dataset.sum(["longitude", "latitude"])
    test_data = year_dataset.sel(time=slice(*TEST_PERIOD))
    train_data = year_dataset.sel(time=slice(*TRAINING_PERIOD))
    preprocessed_test_data = preprocess_train_test_data(test_data)
    preprocessed_train_data = preprocess_train_test_data(train_data)
    return preprocessed_train_data, preprocessed_test_data


def preprocess_data(basins_files_dir):
    basins_files = [f for f in listdir(basins_files_dir) if isfile(join(basins_files_dir, f))]
    df_train = pd.DataFrame(columns=["time", "tp"])
    df_test = pd.DataFrame(columns=["time", "tp"])
    for i in range(len(basins_files)):
        print(f'processing file {i + 1} from {len(basins_files)} files')
        basin_filename = join(basins_files_dir, basins_files[i])
        preprocessed_train_data, preprocessed_test_data = preprocess_single_year_file(basin_filename)
        df_train = pd.concat([df_train, preprocessed_train_data])
        df_test = pd.concat([df_test, preprocessed_test_data])
    df_train = df_train.sort_values(by=["time"], ascending=True)
    df_train.to_csv("./df_train.csv")
    df_test = df_test.sort_values(by=["time"], ascending=True)
    df_test.to_csv("./df_test.csv")


def main():
    boundaries_file_name =  "drive/Computers/My_Laptop_X1/HCDN_nhru_final_671.shp"
    ERA5_data_folder_name = "drive/My Drive/ERA5/"
    output_folder_name = "drive/My Drive/ERA5/"
    # read the basins' boundaries file using gpd.read_file()
    basin_data = gpd.read_file(boundaries_file_name)
    for station_id in basin_data["hru_id"]:
      discharge_file_name = '/Streamflow/dis_' + station_id + '.csv'
      # get the boundaries of the required basin using its station ID
      basin = basin_data[basin_data['hru_id'] == int(station_id)]
      bounds = basin.bounds
      read_single_basin(station_id, bounds, discharge_file_name, ERA5_data_folder_name, output_folder_name)

main()